In [3]:
import pickle
import os

with open(os.path.join("results", "glass", f'results_0.pickle'), 'rb') as f:
    results = pickle.load(f)

results

{'baseline_knn_time': array([0.1540075, 0.0070467, 0.0059965]),
 'baseline_knn_acc': array([0.62790698, 0.69767442, 0.58139535]),
 'baseline_time': array([0.0013885, 0.0009983, 0.0085672, 0.0037015]),
 'baseline_acc': array([0.58139535, 0.53488372, 0.6744186 , 0.72093023]),
 'smart_time': array([[[2.3594546, 0.0590264],
         [0.0597788, 0.0608105],
         [0.0633466, 0.0617818]],
 
        [[0.0539741, 0.0560436],
         [0.0561274, 0.0598776],
         [0.0579494, 0.0598743]],
 
        [[0.0562593, 0.0647282],
         [0.0617295, 0.0715498],
         [0.0676939, 0.0727452]],
 
        [[0.0645766, 0.0715691],
         [0.0708678, 0.0721596],
         [0.0748834, 0.0767115]]]),
 'smart_acc': array([[[0.60465116, 0.6744186 ],
         [0.69767442, 0.6744186 ],
         [0.69767442, 0.69767442]],
 
        [[0.60465116, 0.62790698],
         [0.6744186 , 0.69767442],
         [0.62790698, 0.65116279]],
 
        [[0.62790698, 0.69767442],
         [0.6744186 , 0.6744186 ],
    

In [7]:
import numpy as np
y_neighbors = np.array([0,1,2,0,1,2,0,1,1,1])
k=len(y_neighbors)
treshold = 0.8

unique, counts = np.unique(y_neighbors, return_counts=True)
dominant_class = unique[np.argmax(counts)]
print(unique, counts)

print(counts[np.argmax(counts)], treshold*k)

[0 1 2] [3 5 2]
5 8.0


In [21]:
from typing import Literal
from average_results import ret_avg_results
import numpy as np

def generate_table(datasets=['mnist', 'covertype', 'yeast', 'skin', 'statlog'],knn_algo:Literal['brute', 'kd_tree', 'ball_tree']='brute'):
    ret = ret_avg_results(datasets=datasets, knn_algo=knn_algo)
    if ret is None:
        print("No results found")
        return
    with open('exp1_tab.txt','w') as f:
        for dataset, results_dict in ret.items():
            clfs= results_dict["clfs"]
            ks= results_dict['ks']
            thresholds= results_dict['thresholds']
            t1_idx = thresholds.index(1.0)
            # print(clfs)
            x_baselines = ["{:.1f}".format(xb * 100) for xb in results_dict["baseline_acc"]]
            f.write(f'{dataset}, ks: {ks}\n')
            f.write("\t".join(x_baselines))
            f.write('\n')
            
            smart= results_dict[f"smart_acc"] #shape = (n_clfs, n_k, n_t)
            smart = np.swapaxes(smart, 1, 2) # shape = (n_clfs, n_t, n_k)
            for iclf, clf in enumerate(clfs):
                curr_smart = smart[iclf,t1_idx,:] # shape = (n_k)
                k_le_10 = [idx for idx, k in enumerate(ks) if k<=10]
                mean_acc_k_le_10 = np.mean(curr_smart[k_le_10])
                means = [mean_acc_k_le_10] + [np.mean(curr_smart[idx]) for idx in range(len(ks)) if idx not in k_le_10]
                means = ["{:.1f}".format(mean * 100) for mean in means]
                f.write("\t".join(means))
                f.write("\t")
            f.write('\n')
                
            
        
generate_table(datasets=['covertype', 'glass', 'mnist', 'skin','statlog', 'usps', 'wine', 'yeast'],)